<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/bnfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Loading all the datasets***

In [12]:
# %load_ext google.colab.data_table

In [13]:
!git clone https://github.com/zihadbappy/FakeDatasets.git




fatal: destination path 'FakeDatasets' already exists and is not an empty directory.


In [14]:

import pandas as pd
Dataset1= pd.read_excel('/content/FakeDatasets/FinalData1.xlsx')
# Dataset1.head()

In [15]:
Dataset1.count()
Dataset1.groupby(by='Class').agg('count')

,Title,Statement,Category,Source,Date
Class,,,,,
Fake,993,993,993,993,993
Real,1548,1548,1548,1548,1548


In [16]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [18]:
! cp kaggle.json ~/.kaggle/

In [19]:
! chmod 600 ~/.kaggle/kaggle.json

In [20]:
!kaggle datasets download -d cryptexcode/banfakenews

 80% 49.0M/61.5M [00:01<00:00, 48.2MB/s]
100% 61.5M/61.5M [00:01<00:00, 52.9MB/s]


In [21]:
!unzip banfakenews.zip

Archive:  banfakenews.zip
  inflating: Authentic-48K.csv       
  inflating: Fake-1K.csv             
  inflating: LabeledAuthentic-7K.csv  
  inflating: LabeledFake-1K.csv      


In [22]:
Dataset2= pd.read_csv('/content/LabeledAuthentic-7K.csv')
# Dataset2.tail()

In [23]:
Dataset3= pd.read_csv('/content/LabeledFake-1K.csv')
Dataset3.tail()
Dataset3['label'].dtypes

dtype('int64')

# ***Preparing 1st Dataset***

In [24]:
df1 = pd.DataFrame()

In [25]:
df1= Dataset1.filter(['Title','Statement', 'Class'], axis=1)
df1.loc[(df1.Class=='Real'),'Class']=1
df1.loc[(df1.Class=='Fake'),'Class']=0

In [26]:
df1['Class']=df1['Class'].astype('int')
df1['Class'].dtypes

dtype('int64')

In [27]:
df1.columns=['headline', 'content', 'label']
# df1

In [28]:
df1.groupby('label').agg('count')

,headline,content
label,,
0,993,993
1,1548,1548


# ***Preparing 2nd Dataset***

In [29]:
# Dataset2

In [30]:
df2= Dataset2.filter(['headline','content', 'label'], axis=1)

In [31]:
df2=df2.fillna(1)
df2['label']=df2['label'].astype('int')
df2['label'].dtypes

dtype('int64')

In [32]:
# df2

# ***Preparing 2nd Dataset***

In [33]:
# Dataset3

In [34]:
df3= Dataset3.filter(['headline','content', 'label'], axis=1)

In [35]:
df3=df3.fillna(0)
df3['label']=df3['label'].astype('int')
df3['label'].dtypes

dtype('int64')

In [36]:
# df3

# ***Merging DataFrames***

In [37]:
df=df1
df=df.append(df2,ignore_index=True) 
df=df.append(df3,ignore_index=True) 
df['content'] = df['content'].map(lambda x: x.strip('\n\t'))
df['headline'] = df['headline'].map(lambda x: x.strip('\n\t'))
# df

In [38]:
# df

In [39]:
# print(df)

In [40]:
df.groupby('label').agg('count')

,headline,content
label,,
0,2292,2292
1,8750,8750


# Fixing proportion of fake news and real news

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11042 entries, 0 to 11041
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  11042 non-null  object
 1   content   11042 non-null  object
 2   label     11042 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 258.9+ KB


In [42]:
temp1=df.iloc[:3284]
temp1.groupby('label').agg('count')

,headline,content
label,,
0,993,993
1,2291,2291


In [43]:
temp2= df.iloc[9743:11041]
temp2.groupby('label').agg('count')

,headline,content
label,,
0,1298,1298


In [44]:
df=temp1
df=df.append(temp2)
df.groupby('label').agg('count')

,headline,content
label,,
0,2291,2291
1,2291,2291


In [45]:
import re
from re import sub

In [46]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [47]:
df['headline'] = df.headline.apply(lambda x: preprocessing(str(x)))
df['content'] = df.content.apply(lambda x: preprocessing(str(x)))

In [48]:
df['content'] = df['content'].str.replace(r'\n', ' ')

# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [49]:
df = df.reset_index(drop=True)
# print(df)

In [50]:
pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.9 MB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.0 MB/s 
     |████████████████████████████████| 965 kB 48.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [51]:
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits

btokenizer = BasicTokenizer()

word_list= []
def clean_text(text):
    tokens = btokenizer.tokenize(text)
    filtered = []
    for i in tokens:
        if i in stopwords:
            continue
    
        if i in punctuations + '‘' + '’':
            continue
    
        filtered.append(i)
        word_list.append(i)
    return " ".join(filtered)

# print("********** Before ***************")
# text = df.iloc[1]['content']
# print(text)
# print("\n********** After ***************")
# print(clean_text(text))

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [52]:
df['content'] = df.content.apply(clean_text)


In [53]:
print(len(word_list))

975087


In [54]:
uniq= set(word_list)

In [55]:
print(len(uniq))

96096


In [56]:
from sklearn.feature_extraction.text import CountVectorizer


In [57]:
bow_transformer = CountVectorizer(tokenizer=lambda x: x.split())
bow_transformer.fit(uniq)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function <lambda> at 0x7fa413877050>)

In [58]:
# print(bow_transformer.vocabulary_.iloc[:10])
for value in bow_transformer.vocabulary_:
  print(value)

Streaming output truncated to the last 5000 lines.
কাভার্ড
সফদারকে
ম্যার্কেল
করেছেনমার্কিন
গদির
আকাশপথ
আয়ারল্যান্ডে
বনকে
হরিপাটনী
এনাকোন্ডায়
ফুতকারে
সিগনালের
সম্বৎ
খেয়েছিল
রেড্ডি
ওজস্বী
আমীররে
খেলাটিকে
গণহত্যা
এখাতে
মোবারকের
বলিটা
লাগানোর
মাথাভাঙ্গা
নদীরে
খামচাতে
জলহস্তী
জর্জেটের
বালের
ফিলাপের
মেটৃকে
কাটাকএটা
ক্যাচটি
বিশ্ববিদ্যালয়েও
আমলায়
শিখিয়ে
জীবনু
ব্যায়ামের
অ্যাবসর্বার
মানসিকভাবে
ব্রুয়িং
বেঙ
ফতে
আমেরিকানের
গাড়
উগ্রবাদীরা
জরিনা
লনডনেই
বেক্তিদের
আমীরাত
মেলাঘর
সাবিহাকে
সিংহকেও
কেন্দ্রটিতে
খাইতেছ
সুসংগঠিত
চেলসির
ভাঙ্গিয়া
কাউন্টেস
হয়এর
ঢোলা
বুচ্চেন
আগাছা
অনেকসময়
কাজগুলো
জলকেলি
জেন্টস
চালে
চালকরাও
এসময়
বিজয়তৃষ্ণার
বিশএকুশ
কথাএক
কন্টাক্টের
উদ্দেশ্য
হবেতিা
আরসুলা
রেসিডেন্ট
শীত
মাস্তি
অন্যটাতে
আশরাফমোস্তফা
চেকপয়েন্টে
যাবেনসেখানে
ম্যামোগ্রামের
বাংলাদেশওয়েস্ট
কাউন্টারগুলোর
বুড়ি
লুসাই
সদ্যদের
পশ্চিম
ধারলো
যাউংগা
নিরবে
নীলচে
চেইনের
ভেগাসের
রমমের
গোল
তোড়জোড়
শৃলংকার
মোটরসের
কেয়ারফুলকদিন
পরিশোধও
শিল্পধারা
নেয়ার
কাইকুই
সামর্থ্যকে
দলআবার
ভাবীর
এদিকেই
সহায়ক
বিবাহবিচ্ছেদ
ঘোচুই
জটের
মাদকসহ
কেড়েছে

In [59]:
mess4 = df['content'][3]
print(mess4)

বাইরে বেরোচ্ছি কিছুদিন ভালো বেরোতে পারছি করোনা নামের এক অদ্ভুত অচেনা শত্রু আকস্মিকভাবে ঢুকে পড়েছে জীবনে যুদ্ধ হয়ে বিশ্বজুড়ে অসম যুদ্ধ শত্রুপক্ষ মারাত্মক শক্তিশালী চিনি পাশাপাশি যুদ্ধের বাস্তবতায় শব্দ ঢুকে পড়ছে নিত্যদিনের অভিধানেকরোনা কোয়ারেন্টিন আইসোলেশন সোশ্যাল ডিসট্যান্সিং লকডাউন যেসব শব্দ শুনিওনি সেগুলো দখল নিয়েছে শব্দই শব্দসংশ্লিষ্ট ঘটনার অভিজ্ঞতাও অভূতপূর্ব অভিজ্ঞতা বাংলাদেশেও লকডাউন হওয়ার শব্দটি দিনগুলোও হয়ে কোথাও শব্দ কোলাহল নীরবনির্জন হয়ে সবকিছু জানালা দিয়ে ব্যালকনি দূর যায় দেখি জনমানুষের চিহ্ন কোথাও যানবাহনের শব্দ ফেরিওয়ালার হাঁক মানুষের হাসি এক মৃত নগরী এক জনমানবহীন শহরের স্থির চিত্র বহুকাল এখান মানুষের পদচারণা শেষ হয়ে কেবল ভরদুপুরেও একটা কুকুরের করুণ কান্নার শব্দ শুনে ভেবেছি কোথাও অকল্যাণের আভাস পেয়েছে মা বলতেন যুদ্ধের সময় দুর্ভিক্ষের সময় কুকুর এভাবে কাঁদত দেখছি পোষা বেড়ালটি কিনা পরিবারের সবচেয়ে দুরন্ত সদস্য সারাক্ষণ দুষ্টুমিতে খেলাধুলায় ব্যতিব্যস্ত রাখে ঝিম মেরে শুয়ে থাকছে মন খারাপ সেও পেয়ে মানুষের দুঃসময়ের বার্তা শহরেই কাটল পুরোটা জীবন কখনো দেখিনি কথা একটা ভাইরাসের দাপটে ব

In [142]:
bow4 = bow_transformer.transform([mess4])

In [143]:
print(bow4.nnz)

295


In [146]:
print(bow4.shape)

(1, 96096)


In [147]:
bow_transformer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['©',
 '®এই',
 '»',
 '̈',
 '̈দের',
 'бангладеш',
 'вау',
 'вауবাংলাদেশ',
 'ঃযব',
 'অ',
 'অঁতনি',
 'অঁরি',
 'অঁরিকে',
 'অঁরির',
 'অং',
 'অংক',
 'অংকই',
 'অংকটা',
 'অংকরে',
 'অংকশায়ীনী',
 'অংকিত',
 'অংকে',
 'অংকের',
 'অংগ',
 'অংগজারদারি',
 'অংগন',
 'অংগনে',
 'অংগনের',
 'অংগহানী',
 'অংগীকার',
 'অংগীকারে',
 'অংগুলি',
 'অংগে',
 'অংগের',
 'অংপুর',
 'অংপুরবাসী',
 'অংপুরে',
 'অংপুরের',
 'অংশ',
 'অংশঃ',
 'অংশআল্লাহ',
 'অংশই',
 'অংশএদিকে',
 'অংশও',
 'অংশকে',
 'অংশগুলো',
 'অংশগুলো',
 'অংশগ্রহণ',
 'অংশগ্রহণই',
 'অংশগ্রহণকারী',
 'অংশগ্রহণকারীও',
 'অংশগ্রহণকারীদের',
 'অংশগ্রহণকারীরা',
 'অংশগ্রহণকৃত',
 'অংশগ্রহণমূলক',
 'অংশগ্রহণে',
 'অংশগ্রহণের',
 'অংশগ্রহন',
 'অংশগ্রহনকারিদের',
 'অংশগ্রহনকারিনী',
 'অংশগ্রহনের',
 'অংশজুড়ে',
 'অংশটা',
 'অংশটাই',
 'অংশটি',
 'অংশটিই',
 'অংশটিও',
 'অংশটুকু',
 'অংশটুকুর',
 'অংশবিশেষ',
 'অংশবিশেষকে',
 'অংশসাকা',
 'অংশীজন',
 'অংশীজনদের',
 'অংশীজনের',
 'অংশীদার',
 'অংশীদারও',
 'অংশীদারকী',
 'অংশীদারত্ব',
 'অংশীদারত্বকে',
 'অংশীদারত্বে',
 'অংশীদারদের',
 'অংশীদারিত্ব',
 'অংশ

In [148]:
bow_transformer.get_feature_names()[96000]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'হয়েছেএমন'

In [149]:
messages_bow= bow_transformer.transform(df['content'])

In [150]:
print('Shape of Sparse Matrix: ', messages_bow.shape)


Shape of Sparse Matrix:  (4582, 96096)


In [151]:
messages_bow.nnz

700293

In [153]:
messages_bow.shape

(4582, 96096)

In [154]:
from sklearn.feature_extraction.text import TfidfTransformer


In [155]:
tfidf_transformer= TfidfTransformer().fit(messages_bow)
tfidf4= tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 95944)	0.11087194018106078
  (0, 94545)	0.03942563886824816
  (0, 93486)	0.06517634988208829
  (0, 92631)	0.02272568480515664
  (0, 92075)	0.02669459360226928
  (0, 92057)	0.06750734183723632
  (0, 92014)	0.032241083188280825
  (0, 91763)	0.07079269047588593
  (0, 91760)	0.0485845772220153
  (0, 91554)	0.051740280427286615
  (0, 90832)	0.04013066305707096
  (0, 90105)	0.03951048432122542
  (0, 90026)	0.047150966660805996
  (0, 89962)	0.033083200818293884
  (0, 89655)	0.027701736574889738
  (0, 88429)	0.061891001243438694
  (0, 88012)	0.05000976067198905
  (0, 87318)	0.049124955202851164
  (0, 87299)	0.0483273281006954
  (0, 86787)	0.057751951460690835
  (0, 86527)	0.05345244758872987
  (0, 86330)	0.05000976067198905
  (0, 86327)	0.03457725216468195
  (0, 86204)	0.05697968483846386
  (0, 85634)	0.04593328005515681
  :	:
  (0, 4531)	0.052989312010991454
  (0, 4188)	0.061891001243438694
  (0, 4024)	0.040039106526734564
  (0, 3031)	0.04504197946204579
  (0, 2868)	0.06064196965973385


In [156]:
tfidf_transformer.idf_[bow_transformer.vocabulary_.get('চাল')]


4.3549352692752965

In [157]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [158]:
messages_tfidf.shape

(4582, 96096)

In [159]:
from sklearn.naive_bayes import MultinomialNB

In [160]:
spam_detect_model = MultinomialNB().fit(messages_tfidf, df['label'])


In [169]:
spam_detect_model.predict(tfidf4)[0]

1

In [74]:
all_pred = spam_detect_model.predict(messages_tfidf)

In [184]:
df['label'][1535:1565]

1535    1
1536    1
1537    1
1538    1
1539    1
1540    1
1541    1
1542    1
1543    1
1544    1
1545    1
1546    1
1547    1
1548    0
1549    0
1550    0
1551    0
1552    0
1553    0
1554    0
1555    0
1556    0
1557    0
1558    0
1559    0
1560    0
1561    0
1562    0
1563    0
1564    0
Name: label, dtype: int64

In [185]:
all_pred[1535:1565]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [186]:
from sklearn.model_selection import train_test_split

In [187]:
msg_train, msg_test, label_train, label_test = train_test_split(df['content'], df['label'], test_size=0.4)

In [188]:
from sklearn.pipeline import Pipeline

In [189]:
pipeline = Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())
])

In [190]:
pipeline.fit(msg_train, label_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [191]:
Pred =  pipeline.predict(msg_test)


In [192]:
from sklearn.metrics import classification_report


In [193]:
print(classification_report(label_test, Pred))


              precision    recall  f1-score   support

           0       0.93      0.81      0.87       891
           1       0.84      0.94      0.89       942

    accuracy                           0.88      1833
   macro avg       0.88      0.88      0.88      1833
weighted avg       0.88      0.88      0.88      1833

